In [ ]:
#Only run this in colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/Dimension
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [2]:
import rubics
import numpy as np
from student import student
from classroom import classroom
from teacher import teacher
from rubics_neural_networks import student_network

task = rubics.rubics_task
setup = rubics.rubics_setup

student_network = student_network()
student_template = student(task, 40,student_network)
t = teacher(setup)
c = classroom(task, setup, t, student_template, n_students=1, max_steps=15)
c.run_training_batch(50)


Finished step 0, 48 out of 50 remain open.
Finished step 1, 44 out of 50 remain open.
Finished step 2, 44 out of 50 remain open.
Finished step 3, 44 out of 50 remain open.
Finished step 4, 44 out of 50 remain open.
Finished step 5, 44 out of 50 remain open.
Finished step 6, 44 out of 50 remain open.
Finished step 7, 44 out of 50 remain open.
Finished step 8, 44 out of 50 remain open.
Finished step 9, 44 out of 50 remain open.
Finished step 10, 44 out of 50 remain open.
Finished step 11, 44 out of 50 remain open.
Finished step 12, 44 out of 50 remain open.
Finished step 13, 44 out of 50 remain open.
Finished step 14, 44 out of 50 remain open.
Epoch 1/3
21/21 [==============================] - 4s 8ms/step - loss: 69.7022 - eval_output_loss: 3.4270e-04 - policy_output_loss: 1.5489
Epoch 2/3
21/21 [==============================] - 0s 8ms/step - loss: 19.5360 - eval_output_loss: 2.4663e-04 - policy_output_loss: 0.6383
Epoch 3/3
21/21 [==============================] - 0s 8ms/step - loss: 1